In [1]:
import pandas as pd
import os

In [2]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [3]:
os.chdir("C:/Users/602770/Downloads/wids/Raw Data/")

In [4]:
#Importing the relevant data files 

df_vision = pd.read_csv('Vision_Zero_Safety.csv')
df_pothole1 = pd.read_csv('Pothole__Last_30_Days_08_2019.csv')
df_pothole2 = pd.read_csv('Pothole__Last_30_Days_09_2019.csv')
df_pothole3 = pd.read_csv('Pothole__Last_30_Days_10_2019.csv')
df_bikelane = pd.read_csv('Bicycle_Lanes.csv')

df_lights = pd.read_csv('Street_Lights.csv')
df_crashes = pd.read_csv('Crashes_in_DC.csv')

df_segments = pd.read_csv('Street_Segments.csv')
df_ward = pd.read_csv('Ward_from_2012.csv')

#Need to see if I can find better weather data actually for DC - this file is all MD & VR
#df_weather = pd.read_csv('Weather_Data_DC.csv')

C:\Users\602770\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (16,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\602770\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Vision Zero Cleanup

In [5]:
df_vision.columns

Index(['X', 'Y', 'OBJECTID', 'GLOBALID', 'REQUESTID', 'REQUESTTYPE',
       'REQUESTDATE', 'STATUS', 'STREETSEGID', 'COMMENTS', 'USERTYPE'],
      dtype='object')

In [6]:
len(df_vision)

5566

In [7]:
df_vision.rename(columns={'X':'Longitude', 'Y':'Latitude'}, inplace=True)
#df_vision.columns

In [8]:
df_vision.rename(columns=lambda x:x.lower(), inplace=True)

In [9]:
df_vision.head()

,longitude,latitude,objectid,globalid,requestid,requesttype,requestdate,status,streetsegid,comments,usertype
0,-77.041411,38.916909,1009,{C2EDB76F-5424-40AC-A99C-155A9BE238F9},1009,Red light running,2015-07-02T15:04:43.000Z,Unassigned,12059.0,"Cars continuing on U St east, from Florida Ave...",Pedestrian
1,-76.982451,38.920969,1071,{5E4DD78E-613F-457C-BFAF-C830B03D8B1B},1071,Double parking,2015-07-02T19:57:00.000Z,Unassigned,9521.0,Constant standing around from pedestrians and ...,Car Driver
2,-77.042351,38.987451,1094,{5F6B09D2-DBD0-4664-8CD0-D36A0020B189},1094,Failure to stop for pedestrians,2015-07-02T20:36:55.000Z,Unassigned,8237.0,No sidewalk or crosswalk. And drivers often ma...,Pedestrian
3,-77.042071,38.986620,1095,{1D1F2E6A-8704-4C79-827E-275101347ED2},1095,Other Walking Issue,2015-07-02T20:39:56.000Z,Unassigned,8237.0,Very short sidewalk on the northwest side of i...,Pedestrian
4,-77.048408,38.900703,1105,{BB030B88-2847-481D-A0A8-E7FDA9CA4661},1105,Double parking,2015-07-02T20:55:17.000Z,Unassigned,11728.0,Trucks often double park right at the NE corne...,Pedestrian


In [10]:
df_vision.columns

Index(['longitude', 'latitude', 'objectid', 'globalid', 'requestid',
       'requesttype', 'requestdate', 'status', 'streetsegid', 'comments',
       'usertype'],
      dtype='object')

In [11]:
df_vision = df_vision.loc[:,['objectid', 'latitude','longitude', 'requesttype',
                             'requestdate', 'streetsegid','comments','usertype']]

In [12]:
df_vision.head()

,objectid,latitude,longitude,requesttype,requestdate,streetsegid,comments,usertype
0,1009,38.916909,-77.041411,Red light running,2015-07-02T15:04:43.000Z,12059.0,"Cars continuing on U St east, from Florida Ave...",Pedestrian
1,1071,38.920969,-76.982451,Double parking,2015-07-02T19:57:00.000Z,9521.0,Constant standing around from pedestrians and ...,Car Driver
2,1094,38.987451,-77.042351,Failure to stop for pedestrians,2015-07-02T20:36:55.000Z,8237.0,No sidewalk or crosswalk. And drivers often ma...,Pedestrian
3,1095,38.986620,-77.042071,Other Walking Issue,2015-07-02T20:39:56.000Z,8237.0,Very short sidewalk on the northwest side of i...,Pedestrian
4,1105,38.900703,-77.048408,Double parking,2015-07-02T20:55:17.000Z,11728.0,Trucks often double park right at the NE corne...,Pedestrian


In [13]:
df_vision['requesttype'].unique()

array(['Red light running', 'Double parking',
       'Failure to stop for pedestrians', 'Other Walking Issue',
       'Other Biking Issue', 'Blocking the crosswalk', 'Poor visibility',
       'Not enough time to cross', 'Stop sign running',
       'Other Driving Issue', 'Blocking the bikebox', 'Speeding',
       'Jaywalking', 'Long wait to cross', 'Accessibility Issue',
       'Cyclist behavior', 'Long distance to cross'], dtype=object)

In [14]:
len(df_vision)

5566

In [15]:
needed = ["Speeding", "Other Driving Issue",  "Other Biking Issue", 
          "Blocking the bikebox", "Red light running", 
          "Poor visibility", "Stop sign running", "Double parking", 
          "majorinjuries_bicyclist","minorinjuries_bicyclist",
          "fatal_bicyclist", "bicyclistsimpaired"
         ]

In [16]:
#Filtering for the request types that involve safety of bicyclists as determined after reading through comments
# Can also use to get the ones NOT in the given list of types
        #df_vision[~df_vision['requesttype'].isin(needed)]
    
df_bike_vision = df_vision[df_vision['requesttype'].isin(needed)]

In [17]:
df_bike_vision.head()

,objectid,latitude,longitude,requesttype,requestdate,streetsegid,comments,usertype
0,1009,38.916909,-77.041411,Red light running,2015-07-02T15:04:43.000Z,12059.0,"Cars continuing on U St east, from Florida Ave...",Pedestrian
1,1071,38.920969,-76.982451,Double parking,2015-07-02T19:57:00.000Z,9521.0,Constant standing around from pedestrians and ...,Car Driver
4,1105,38.900703,-77.048408,Double parking,2015-07-02T20:55:17.000Z,11728.0,Trucks often double park right at the NE corne...,Pedestrian
6,1097,38.895371,-77.033648,Other Biking Issue,2015-07-02T20:58:54.000Z,759.0,The 15th street cycletrack should be extended ...,Biker
7,1106,38.885112,-77.059171,Other Biking Issue,2015-07-02T21:01:40.000Z,1228.0,Crossing the GW Parkway shouldn't be done at-g...,Biker


In [18]:
len(df_bike_vision)

3420

## Pothole Cleanup & Consolidation

In [19]:
df_pothole1.columns

Index(['X', 'Y', 'OBJECTID', 'SERVICECODE', 'SERVICECODEDESCRIPTION',
       'SERVICETYPECODEDESCRIPTION', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT',
       'ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE',
       'INSPECTIONFLAG', 'INSPECTIONDATE', 'INSPECTORNAME',
       'SERVICEORDERSTATUS', 'STATUS_CODE', 'SERVICEREQUESTID', 'PRIORITY',
       'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'CITY',
       'STATE', 'ZIPCODE', 'MARADDRESSREPOSITORYID', 'WARD'],
      dtype='object')

In [20]:
df_pothole1.head()

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,SERVICEDUEDATE,SERVICEORDERDATE,INSPECTIONFLAG,INSPECTIONDATE,INSPECTORNAME,SERVICEORDERSTATUS,STATUS_CODE,SERVICEREQUESTID,PRIORITY,STREETADDRESS,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD
0,-77.018106,38.822447,1,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-08T15:59:43.000Z,2019-08-09T07:48:15.000Z,2019-08-13T15:59:43.000Z,2019-08-08T15:59:43.000Z,0.0,2019-08-09T07:48:14.000Z,NaN,Closed,NaN,19-00827099,Standard,4555 OVERLOOK AVENUE SW,398427.910000,128295.650000,38.822440,-77.018104,WASHINGTON,DC,20375,291850,8.0
1,-77.057234,38.935178,2,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-10T20:21:17.000Z,2019-08-12T18:55:04.000Z,2019-08-14T20:21:17.000Z,2019-08-10T20:21:17.000Z,0.0,2019-08-12T18:55:03.000Z,NaN,Closed,NaN,19-00830369,Standard,2738 ORDWAY STREET NW,395037.950000,140811.020000,38.935170,-77.057232,WASHINGTON,DC,20008,223253,3.0
2,-76.946425,38.867561,3,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-12T15:20:44.000Z,2019-08-13T16:25:11.000Z,2019-08-15T15:20:44.000Z,2019-08-12T15:20:44.000Z,0.0,2019-08-13T16:25:10.000Z,NaN,Closed,NaN,19-00832690,Standard,1600 FORT DAVIS PLACE SE,404649.600000,133304.770000,38.867553,-76.946423,WASHINGTON,DC,20020,60539,7.0
3,-77.045492,38.917941,4,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-06T16:43:09.000Z,2019-08-08T16:43:22.000Z,2019-08-09T16:43:09.000Z,2019-08-06T16:43:09.000Z,0.0,2019-08-08T16:43:20.000Z,NaN,Closed,NaN,19-00822278,Standard,20TH STREET NW AND COLUMBIA ROAD NW,396055.031258,138897.052649,38.917934,-77.045490,WASHINGTON,DC,20009,900109,1.0
4,-77.075338,38.963316,5,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-19T08:31:15.000Z,2019-08-22T09:15:12.000Z,2019-08-22T08:31:15.000Z,2019-08-19T08:31:15.000Z,0.0,2019-08-22T09:15:11.000Z,NaN,Closed,NaN,19-00844178,Standard,3815 LIVINGSTON STREET NW,393470.880000,143935.700000,38.963308,-77.075336,WASHINGTON,DC,20015,284799,3.0


In [21]:
df_pothole1.drop(axis=1, columns=['X','Y', 'XCOORD', 'YCOORD'], inplace=True)
df_pothole2.drop(axis=1, columns=['X','Y', 'XCOORD', 'YCOORD'], inplace=True)
df_pothole3.drop(axis=1, columns=['X','Y', 'XCOORD', 'YCOORD'], inplace=True)

In [22]:
df_pothole1.rename(columns=lambda x: x.lower(), inplace=True)
df_pothole2.rename(columns=lambda x: x.lower(), inplace=True)
df_pothole3.rename(columns=lambda x: x.lower(), inplace=True)

In [23]:
df_pothole1.head()

,objectid,servicecode,servicecodedescription,servicetypecodedescription,organizationacronym,servicecallcount,adddate,resolutiondate,serviceduedate,serviceorderdate,inspectionflag,inspectiondate,inspectorname,serviceorderstatus,status_code,servicerequestid,priority,streetaddress,latitude,longitude,city,state,zipcode,maraddressrepositoryid,ward
0,1,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-08T15:59:43.000Z,2019-08-09T07:48:15.000Z,2019-08-13T15:59:43.000Z,2019-08-08T15:59:43.000Z,0.0,2019-08-09T07:48:14.000Z,NaN,Closed,NaN,19-00827099,Standard,4555 OVERLOOK AVENUE SW,38.822440,-77.018104,WASHINGTON,DC,20375,291850,8.0
1,2,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-10T20:21:17.000Z,2019-08-12T18:55:04.000Z,2019-08-14T20:21:17.000Z,2019-08-10T20:21:17.000Z,0.0,2019-08-12T18:55:03.000Z,NaN,Closed,NaN,19-00830369,Standard,2738 ORDWAY STREET NW,38.935170,-77.057232,WASHINGTON,DC,20008,223253,3.0
2,3,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-12T15:20:44.000Z,2019-08-13T16:25:11.000Z,2019-08-15T15:20:44.000Z,2019-08-12T15:20:44.000Z,0.0,2019-08-13T16:25:10.000Z,NaN,Closed,NaN,19-00832690,Standard,1600 FORT DAVIS PLACE SE,38.867553,-76.946423,WASHINGTON,DC,20020,60539,7.0
3,4,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-06T16:43:09.000Z,2019-08-08T16:43:22.000Z,2019-08-09T16:43:09.000Z,2019-08-06T16:43:09.000Z,0.0,2019-08-08T16:43:20.000Z,NaN,Closed,NaN,19-00822278,Standard,20TH STREET NW AND COLUMBIA ROAD NW,38.917934,-77.045490,WASHINGTON,DC,20009,900109,1.0
4,5,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-19T08:31:15.000Z,2019-08-22T09:15:12.000Z,2019-08-22T08:31:15.000Z,2019-08-19T08:31:15.000Z,0.0,2019-08-22T09:15:11.000Z,NaN,Closed,NaN,19-00844178,Standard,3815 LIVINGSTON STREET NW,38.963308,-77.075336,WASHINGTON,DC,20015,284799,3.0


In [24]:
df_pothole2.head()

,objectid,servicecode,servicecodedescription,servicetypecodedescription,organizationacronym,servicecallcount,adddate,resolutiondate,serviceduedate,serviceorderdate,inspectionflag,inspectiondate,inspectorname,serviceorderstatus,status_code,servicerequestid,priority,streetaddress,latitude,longitude,city,state,zipcode,maraddressrepositoryid,ward
0,1,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-09-23T13:36:57.000Z,2019-09-24T14:00:12.000Z,2019-09-26T13:36:57.000Z,2019-09-23T13:36:57.000Z,0.0,2019-09-24T14:00:11.000Z,NaN,Closed,NaN,19-00911116,Standard,915 5TH STREET NW,38.901865,-77.018690,WASHINGTON,DC,20001,237568,6.0
1,2,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-09-19T17:21:26.000Z,2019-09-24T17:00:13.000Z,2019-09-24T17:21:26.000Z,2019-09-19T17:21:26.000Z,0.0,2019-09-24T17:00:11.000Z,NaN,Closed,NaN,19-00906371,Standard,19TH STREET SE AND S STREET SE,38.868236,-76.977355,WASHINGTON,DC,20020,902775,8.0
2,3,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-09-19T17:28:37.000Z,2019-09-25T16:50:05.000Z,2019-09-24T17:28:37.000Z,2019-09-19T17:28:37.000Z,0.0,2019-09-25T16:50:03.000Z,NaN,Closed,NaN,19-00906387,Standard,1375 K STREET NW,38.902771,-77.031665,WASHINGTON,DC,20005,304721,2.0
3,4,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-09-18T13:02:10.000Z,2019-09-23T18:55:06.000Z,2019-09-23T13:02:09.000Z,2019-09-18T13:02:10.000Z,0.0,2019-09-23T18:55:05.000Z,NaN,Closed,NaN,19-00903117,Standard,3106 GEORGIA AVENUE NW,38.929344,-77.023646,WASHINGTON,DC,20010,232892,1.0
4,5,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-09-18T09:09:22.000Z,2019-09-24T13:50:18.000Z,2019-09-23T09:09:22.000Z,2019-09-18T09:09:22.000Z,0.0,2019-09-24T13:50:17.000Z,NaN,Closed,NaN,19-00902200,Standard,214 TAYLOR STREET NW,38.940900,-77.015159,WASHINGTON,DC,20011,244287,4.0


In [25]:
len(df_pothole1)

397

In [26]:
len(df_pothole2)

246

In [27]:
len(df_pothole3)

273

In [28]:
df_potholes = pd.concat([df_pothole1, df_pothole2, df_pothole3])

In [29]:
len(df_potholes)

916

In [30]:
df_potholes.columns

Index(['objectid', 'servicecode', 'servicecodedescription',
       'servicetypecodedescription', 'organizationacronym', 'servicecallcount',
       'adddate', 'resolutiondate', 'serviceduedate', 'serviceorderdate',
       'inspectionflag', 'inspectiondate', 'inspectorname',
       'serviceorderstatus', 'status_code', 'servicerequestid', 'priority',
       'streetaddress', 'latitude', 'longitude', 'city', 'state', 'zipcode',
       'maraddressrepositoryid', 'ward'],
      dtype='object')

In [31]:
df_potholes.head()

,objectid,servicecode,servicecodedescription,servicetypecodedescription,organizationacronym,servicecallcount,adddate,resolutiondate,serviceduedate,serviceorderdate,inspectionflag,inspectiondate,inspectorname,serviceorderstatus,status_code,servicerequestid,priority,streetaddress,latitude,longitude,city,state,zipcode,maraddressrepositoryid,ward
0,1,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-08T15:59:43.000Z,2019-08-09T07:48:15.000Z,2019-08-13T15:59:43.000Z,2019-08-08T15:59:43.000Z,0.0,2019-08-09T07:48:14.000Z,NaN,Closed,NaN,19-00827099,Standard,4555 OVERLOOK AVENUE SW,38.822440,-77.018104,WASHINGTON,DC,20375,291850,8.0
1,2,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-10T20:21:17.000Z,2019-08-12T18:55:04.000Z,2019-08-14T20:21:17.000Z,2019-08-10T20:21:17.000Z,0.0,2019-08-12T18:55:03.000Z,NaN,Closed,NaN,19-00830369,Standard,2738 ORDWAY STREET NW,38.935170,-77.057232,WASHINGTON,DC,20008,223253,3.0
2,3,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-12T15:20:44.000Z,2019-08-13T16:25:11.000Z,2019-08-15T15:20:44.000Z,2019-08-12T15:20:44.000Z,0.0,2019-08-13T16:25:10.000Z,NaN,Closed,NaN,19-00832690,Standard,1600 FORT DAVIS PLACE SE,38.867553,-76.946423,WASHINGTON,DC,20020,60539,7.0
3,4,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-06T16:43:09.000Z,2019-08-08T16:43:22.000Z,2019-08-09T16:43:09.000Z,2019-08-06T16:43:09.000Z,0.0,2019-08-08T16:43:20.000Z,NaN,Closed,NaN,19-00822278,Standard,20TH STREET NW AND COLUMBIA ROAD NW,38.917934,-77.045490,WASHINGTON,DC,20009,900109,1.0
4,5,S0301,Pothole,Toa-Street & Bridge Maintenance,DDOT,1,2019-08-19T08:31:15.000Z,2019-08-22T09:15:12.000Z,2019-08-22T08:31:15.000Z,2019-08-19T08:31:15.000Z,0.0,2019-08-22T09:15:11.000Z,NaN,Closed,NaN,19-00844178,Standard,3815 LIVINGSTON STREET NW,38.963308,-77.075336,WASHINGTON,DC,20015,284799,3.0


In [32]:
df_potholes = df_potholes.loc[:,['objectid', 'latitude', 'longitude', 'adddate', 'resolutiondate', 
                                 'streetaddress', 'zipcode', 'ward' ]]

In [33]:
df_potholes.rename(columns={'streetaddress':'street_address'}, inplace=True)

In [34]:
df_potholes.head()

,objectid,latitude,longitude,adddate,resolutiondate,street_address,zipcode,ward
0,1,38.822440,-77.018104,2019-08-08T15:59:43.000Z,2019-08-09T07:48:15.000Z,4555 OVERLOOK AVENUE SW,20375,8.0
1,2,38.935170,-77.057232,2019-08-10T20:21:17.000Z,2019-08-12T18:55:04.000Z,2738 ORDWAY STREET NW,20008,3.0
2,3,38.867553,-76.946423,2019-08-12T15:20:44.000Z,2019-08-13T16:25:11.000Z,1600 FORT DAVIS PLACE SE,20020,7.0
3,4,38.917934,-77.045490,2019-08-06T16:43:09.000Z,2019-08-08T16:43:22.000Z,20TH STREET NW AND COLUMBIA ROAD NW,20009,1.0
4,5,38.963308,-77.075336,2019-08-19T08:31:15.000Z,2019-08-22T09:15:12.000Z,3815 LIVINGSTON STREET NW,20015,3.0


## Bikelane Cleanup

In [35]:
df_bikelane.head()

,OBJECTID,FACILITYID,STREETSEGID,SOURCEID,BIKELANELENGTH,FACILITY,PROPOSEDCYCLETRACK,Shape_Length,TRAVELDIRECTION,NOTES,BIKELANE_YEAR,PLANSREADY,GAP,GAP_NOTES,NEED_SYMBOL,NEED_SYM_1,REPAINT_LINE,YEAR_INSTALLED
0,1,SEGID- 9854,9854,15135550,0.009455,Existing Bike Lane,,15.216846,,,0,,,,NaN,NaN,NaN,2008
1,2,SEGID- 10198,10198,15135555,0.076754,Existing Bike Lane,,123.524976,,"COG 95, 1975 plan, public input",0,,,,NaN,NaN,NaN,2008
2,3,SEGID- 9167,9167,15135560,0.058610,Existing Bike Lane,,94.322895,,"COG 95, 1975 plan, public input",0,,,,NaN,NaN,NaN,2008
3,4,SEGID- 12337,12337,15135565,0.026891,Existing Bike Lane,,43.276471,,"COG 95, 1975 plan, public input",0,,,,NaN,NaN,NaN,2008
4,5,SEGID- 10611,10611,15135580,0.172385,Existing Bike Lane,,277.425702,,"COG 95, 1975 plan, public input",0,,,,NaN,NaN,NaN,2008


In [36]:
df_bikelane.rename(columns=lambda x:x.lower(), inplace=True)

In [37]:
df_bikelane.columns

Index(['objectid', 'facilityid', 'streetsegid', 'sourceid', 'bikelanelength',
       'facility', 'proposedcycletrack', 'shape_length', 'traveldirection',
       'notes', 'bikelane_year', 'plansready', 'gap', 'gap_notes',
       'need_symbol', 'need_sym_1', 'repaint_line', 'year_installed'],
      dtype='object')

In [38]:
df_bikelane = df_bikelane.loc[:,['objectid','facilityid', 'streetsegid', 'sourceid',
                                 'bikelanelength','facility', 'shape_length', ]]

In [39]:
df_bikelane.head()

,objectid,facilityid,streetsegid,sourceid,bikelanelength,facility,shape_length
0,1,SEGID- 9854,9854,15135550,0.009455,Existing Bike Lane,15.216846
1,2,SEGID- 10198,10198,15135555,0.076754,Existing Bike Lane,123.524976
2,3,SEGID- 9167,9167,15135560,0.058610,Existing Bike Lane,94.322895
3,4,SEGID- 12337,12337,15135565,0.026891,Existing Bike Lane,43.276471
4,5,SEGID- 10611,10611,15135580,0.172385,Existing Bike Lane,277.425702


## Lights Cleanup

In [40]:
df_lights.head()

,X,Y,ADDEDBY,ADDTOGIS,ARMLENGTH1_DESC,ARMLENGTH2_DESC,ARMSTYLE_DESC,ASSETSTATUS_DESC,ASSETTYPE,CCT_DESC,CONDITION_DESC,CROSSSTREET,FACILITYID,FEEDMANHOLE_DESC,FIXTURESTYLE_DESC,GLOBALID,HOUSENO,INOPERATION,ISMETERED_DESC,ISMODIFIED_DESC,LASTMODIFIED,LASTPAINTED,LEDINOPERATION,LIGHTHISTORY,LIGHTMANUFACTURER_DESC,LIGHTTYPE_DESC,NUMBERLIGHTS,NUMBERARMS_DESC,OBJECTID,OTHEREQUIPMENT,OWNER_DESC,PEPCOLIGHTID,POLECOLOR_DESC,POLECOMPOSITION_DESC,POLEHEIGHT_DESC,POLETYPE_DESC,POWERFEED_DESC,PROXIMITY_DESC,QUADRANT_DESC,RMS_DESC,ROADCLASSIFICATION_DESC,SHIELD_DESC,STREETLIGHTID,STREETNAME,STREETSEGMID,TBASETYPE_DESC,TRAFFICCOMBO,WARD,WATTAGE1,WATTAGE2,WHATMODIFIED,WHYINACTIVE
0,-77.035705,38.901943,Osmose,2007-07-23T00:00:00.000Z,3.0,NaN,Regular Alley,Active,Alley Light,NaN,NaN,16TH ST,23289,NaN,Cobrahead Semi Cutoff,{3AFC04D6-52E5-4285-A932-48986DB3FE5F},1528,NaN,Not Metered,NO,NaN,NaN,NaN,7/13/2016 Correct asset location.,General Electric,HPS-High Pressure Sodium,1,1.0,7693882,NO,DDOT,NaN,NaN,Steel,28 ft,5A,Underground,Rear,NW,Not Controlled,Local,None,23289,K ST,555,None,NO,2,100.0,NaN,NaN,NaN
1,-76.985184,38.869033,Osmose,2007-04-18T00:00:00.000Z,6.0,NaN,Simple Pendant,Active,Street Light,NaN,NaN,RIDGE PL,80541,NaN,Cobrahead Semi Cutoff,{BEE02180-79F1-467E-9CDC-77BD5E7D2D76},1714,NaN,Not Metered,NO,NaN,NaN,NaN,10/6/2016 Correct asset location.,General Electric,HPS-High Pressure Sodium,1,1.0,7693883,NO,DDOT,NaN,Gray,Steel,30 ft,PP,Underground,Opposite,SE,Not Controlled,Local,None,80541,14TH ST,1661,Non-Breakaway,NO,8,250.0,NaN,NaN,NaN
2,-76.979626,38.863385,Osmose,2007-04-19T00:00:00.000Z,3.0,NaN,Regular Alley,Active,Alley Light,NaN,NaN,18TH ST,80760DC,NaN,Cobrahead Semi Cutoff,{9953838D-699A-4E66-A0C9-F92D8DB735F4},1711,NaN,Not Metered,NO,NaN,NaN,NaN,1/28/2016 Correct asset location.,General Electric,HPS-High Pressure Sodium,1,1.0,7693884,NO,PEPCO,805375-7307,NaN,Wood,35 ft,Wood,Overhead,Rear,SE,Not Controlled,Local,None,80760DC,V ST,2883,None,NO,8,150.0,NaN,NaN,NaN
3,-76.971740,38.872472,Osmose,2007-04-11T00:00:00.000Z,3.0,NaN,Regular Alley,Active,Alley Light,NaN,NaN,MINNESOTA AVE,80485DC,NaN,Cobrahead Cutoff,{D8668E48-98A1-4E94-97AD-02ADFD94AF94},1506,NaN,Not Metered,NO,NaN,NaN,NaN,10/15/2015 Correct asset location.,General Electric,HPS-High Pressure Sodium,1,1.0,7693885,NO,Verizon,NaN,NaN,Wood,35 ft,Wood,Overhead,Side,SE,Not Controlled,Local,None,80485DC,WHITE PL,2561,None,NO,8,150.0,NaN,NaN,NaN
4,-76.974904,38.868505,Osmose,2007-04-10T00:00:00.000Z,6.0,NaN,WP Arm with Guy,Active,Street Light,NaN,NaN,S ST,80277DC,NaN,Cobrahead Cutoff,{8B705253-694E-495C-AFD3-2741B0A34030},1722,NaN,Not Metered,NO,NaN,NaN,NaN,10/14/2015 Correct asset location.,General Electric,HPS-High Pressure Sodium,1,1.0,7693886,NO,PEPCO,807376-0794,NaN,Wood,35 ft,Wood,Overhead,Front,SE,Not Controlled,Local,None,80277DC,22ND ST,2645,None,NO,8,250.0,NaN,NaN,NaN


In [41]:
df_lights.rename(columns=lambda x:x.lower(), inplace=True)

In [42]:
# df_lights['NUMBERLIGHTS'].value_counts()

In [43]:
df_lights.columns

Index(['x', 'y', 'addedby', 'addtogis', 'armlength1_desc', 'armlength2_desc',
       'armstyle_desc', 'assetstatus_desc', 'assettype', 'cct_desc',
       'condition_desc', 'crossstreet', 'facilityid', 'feedmanhole_desc',
       'fixturestyle_desc', 'globalid', 'houseno', 'inoperation',
       'ismetered_desc', 'ismodified_desc', 'lastmodified', 'lastpainted',
       'ledinoperation', 'lighthistory', 'lightmanufacturer_desc',
       'lighttype_desc', 'numberlights', 'numberarms_desc', 'objectid',
       'otherequipment', 'owner_desc', 'pepcolightid', 'polecolor_desc',
       'polecomposition_desc', 'poleheight_desc', 'poletype_desc',
       'powerfeed_desc', 'proximity_desc', 'quadrant_desc', 'rms_desc',
       'roadclassification_desc', 'shield_desc', 'streetlightid', 'streetname',
       'streetsegmid', 'tbasetype_desc', 'trafficcombo', 'ward', 'wattage1',
       'wattage2', 'whatmodified', 'whyinactive'],
      dtype='object')

In [44]:
df_lights = df_lights.loc[:, ['x','y', 'numberlights','streetname', 'streetsegmid', 'ward']]

In [45]:
df_lights.rename(columns={'x':'Longitude', 'y':'Latitude'}, inplace=True)

In [46]:
df_lights.head()

,Longitude,Latitude,numberlights,streetname,streetsegmid,ward
0,-77.035705,38.901943,1,K ST,555,2
1,-76.985184,38.869033,1,14TH ST,1661,8
2,-76.979626,38.863385,1,V ST,2883,8
3,-76.971740,38.872472,1,WHITE PL,2561,8
4,-76.974904,38.868505,1,22ND ST,2645,8


## Ward Cleanup

In [47]:
df_ward

,OBJECTID,WARD,NAME,REP_NAME,WEB_URL,REP_PHONE,REP_EMAIL,REP_OFFICE,WARD_ID,LABEL,AREASQMI,Shape_Length,Shape_Area,POP_2000,POP_2010,POP_2011_2015,POP_BLACK,POP_NATIVE_AMERICAN,POP_ASIAN,POP_HAWAIIAN,POP_OTHER_RACE,TWO_OR_MORE_RACES,NOT_HISPANIC_OR_LATINO,HISPANIC_OR_LATINO,POP_MALE,POP_FEMALE,AGE_0_5,AGE_5_9,AGE_10_14,AGE_15_17,AGE_18_19,AGE_20,AGE_21,AGE_22_24,AGE_25_29,AGE_30_34,AGE_35_39,AGE_40_44,AGE_45_49,AGE_50_54,AGE_55_59,AGE_60_61,AGE_65_66,AGE_67_69,AGE_70_74,AGE_75_79,AGE_80_84,AGE_85_PLUS,MEDIAN_AGE,UNEMPLOYMENT_RATE,TOTAL_HH,FAMILY_HH,PCT_FAMILY_HH,NONFAMILY_HH,PCT_NONFAMILY_HH,PCT_BELOW_POV,PCT_BELOW_POV_FAM,PCT_BELOW_POV_WHITE,PCT_BELOW_POV_BLACK,PCT_BELOW_POV_NAT_AMER,PCT_BELOW_POV_ASIAN,PCT_BELOW_POV_HAWAIIAN,PCT_BELOW_POV_OTHER,PCT_BELOW_POV_TWO_RACES,POP_25_PLUS,POP_25_PLUS_9TH_GRADE,POP_25_PLUS_GRADUATE,MARRIED_COUPLE_FAMILY,MALE_HH_NO_WIFE,FEMALE_HH_NO_HUSBAND,MEDIAN_HH_INCOME,PER_CAPITA_INCOME,PCT_BELOW_POV_HISP,PCT_BELOW_POV_WHTE_NOHISP,NO_DIPLOMA_25_PLUS,DIPLOMA_25_PLUS,NO_DEGREE_25_PLUS,ASSOC_DEGREE_25_PLUS,BACH_DEGREE_25_PLUS,MED_VAL_OOU,Shape_Length0,Shape_Area0
0,1,8,Ward 8,"Trayon White, Sr.",http://dccouncil.us/council/trayon-white-sr,(202) 724-8045,twhite@dccouncil.us,"1350 Pennsylvania Ave, Suite 400, NW 20004",8,Ward 8,11.937871,28714.069430,3.096585e+07,74049,73662,81133,75259,110,310,12,711,872,79843,1290,35573,45560,7879,7061,5963,3596,2945,1800,1748,4070,6306,5951,4617,4873,4429,4978,5001,1578,1011,1211,1904,1026,634,432,29.3,22.9,29470,17747,60.220563,11723,39.779437,37.7,35.3,10.8,38.8,54.1,21.6,0,51.2,36.4,NaN,1858,2614,NaN,1886,11653,30910,17596,36.3,10.8,5958,18736,10975,2149,3781,229900,28714.069430,3.096585e+07
1,2,6,Ward 6,Charles Allen,http://dccouncil.us/council/charles-allen,(202) 724-8072,callen@dccouncil.us,"1350 Pennsylvania Ave, Suite 406, NW 20004",6,Ward 6,6.221045,24157.977435,1.606492e+07,70867,76238,84290,29909,295,3573,40,1233,2529,79000,5290,40411,43879,4779,2747,2235,1088,1370,794,724,4596,13427,12512,8052,5474,4524,4350,4724,1830,1518,1596,2167,1522,816,958,33.9,6.3,40100,15110,37.680798,24990,62.319202,12.5,9.6,4.5,25.7,13.9,11.6,0,10.7,6.2,NaN,1785,25882,NaN,944,4157,94343,58354,6.3,4.5,3128,7079,6643,1852,19588,573200,24157.977435,1.606492e+07
2,3,7,Ward 7,Vincent Gray,http://dccouncil.us/council/vincent-gray,(202) 724-8068,vgray@dccouncil.us,"1350 Pennsylvania Ave, Suite 406, NW 20004",7,Ward 7,8.809914,22345.226917,2.281818e+07,69987,71748,73290,69005,219,225,17,1211,908,70987,2303,33916,39374,5230,4485,4333,2944,2162,1347,1107,3170,5036,5083,4154,5166,4794,5715,5272,1716,1254,1680,2370,1719,1290,1142,37.0,19.1,29266,15574,53.215335,13692,46.784665,27.2,23.6,8.9,28.0,12.8,5.3,0,19.7,20.0,NaN,2259,3309,NaN,1682,9499,39165,22917,22.9,8.9,6002,18683,10800,2569,4890,238900,22345.226917,2.281818e+07
3,4,2,Ward 2,Jack Evans,http://dccouncil.us/council/jack-evans,(202) 724-8058,jevans@dccouncil.us,"1350 Pennsylvania Ave, Suite 106, NW 20004",2,Ward 2,8.684517,29545.803886,2.249280e+07,63455,76645,77645,6817,213,7640,30,2496,2875,69529,8116,39214,38431,2173,1110,571,486,6200,3363,3485,6463,12614,10475,6491,4175,3445,3413,3248,1447,1214,1325,1763,764,724,863,30.9,3.7,38870,9071,23.336764,29799,76.663236,13.4,4.7,10.1,33.2,26.0,23.5,0,12.6,10.3,NaN,1743,28509,NaN,458,754,100388,72388,13.5,10.1,988,2381,2980,940,16253,623500,29545.803886,2.249280e+07
4,5,1,Ward 1,Brianne Nadeau,http://dccouncil.us/council/brianne-nadeau,(202) 724-8181,bnadeau@dccouncil.us,"1350 Pennsylvania Ave, Suite 102, NW 20004",1,Ward 1,2.535896,12925.384774,6.567941e+06,71747,74462,82859,25110,300,3509,111,6259,2596,65755,17104,41368,41491,4733,2644,1934,1223,2399,1613,1642,5487,15600,12286,7378,5099,4784,3963,2618,1420,802,1208,1778,954,505,738,31.3,6.6,34907,12253,35.101842,22654,64.898158,13.5,11.0,7.7,23.3,8.7,6.9,0,26.8,11.5,NaN,4324,22488,NaN,1634,3133,82159,47982,17.9,7.7,3224,6984,5293,1258,17613,542100,12925.384774,6.567941e+06
5,6,5,Ward 5,Kenyan McDuffie,http://dccouncil.us/council/kenyan

In [48]:
df_ward.columns

Index(['OBJECTID', 'WARD', 'NAME', 'REP_NAME', 'WEB_URL', 'REP_PHONE',
       'REP_EMAIL', 'REP_OFFICE', 'WARD_ID', 'LABEL', 'AREASQMI',
       'Shape_Length', 'Shape_Area', 'POP_2000', 'POP_2010', 'POP_2011_2015',
       'POP_BLACK', 'POP_NATIVE_AMERICAN', 'POP_ASIAN', 'POP_HAWAIIAN',
       'POP_OTHER_RACE', 'TWO_OR_MORE_RACES', 'NOT_HISPANIC_OR_LATINO',
       'HISPANIC_OR_LATINO', 'POP_MALE', 'POP_FEMALE', 'AGE_0_5', 'AGE_5_9',
       'AGE_10_14', 'AGE_15_17', 'AGE_18_19', 'AGE_20', 'AGE_21', 'AGE_22_24',
       'AGE_25_29', 'AGE_30_34', 'AGE_35_39', 'AGE_40_44', 'AGE_45_49',
       'AGE_50_54', 'AGE_55_59', 'AGE_60_61', 'AGE_65_66', 'AGE_67_69',
       'AGE_70_74', 'AGE_75_79', 'AGE_80_84', 'AGE_85_PLUS', 'MEDIAN_AGE',
       'UNEMPLOYMENT_RATE', 'TOTAL_HH', 'FAMILY_HH', 'PCT_FAMILY_HH',
       'NONFAMILY_HH', 'PCT_NONFAMILY_HH', 'PCT_BELOW_POV',
       'PCT_BELOW_POV_FAM', 'PCT_BELOW_POV_WHITE', 'PCT_BELOW_POV_BLACK',
       'PCT_BELOW_POV_NAT_AMER', 'PCT_BELOW_POV_ASIAN',
      

In [49]:
df_ward = df_ward.loc[:, ['OBJECTID', 'WARD', 'AREASQMI', 'Shape_Length', 'Shape_Area',]]

In [50]:
df_ward.head()

,OBJECTID,WARD,AREASQMI,Shape_Length,Shape_Area
0,1,8,11.937871,28714.069430,3.096585e+07
1,2,6,6.221045,24157.977435,1.606492e+07
2,3,7,8.809914,22345.226917,2.281818e+07
3,4,2,8.684517,29545.803886,2.249280e+07
4,5,1,2.535896,12925.384774,6.567941e+06


## Crashes Cleanup

In [51]:
df_crashes.head()

,X,Y,OBJECTID,CRIMEID,CCN,REPORTDATE,ROUTEID,MEASURE,OFFSET,STREETSEGID,ROADWAYSEGID,FROMDATE,TODATE,MARID,ADDRESS,LATITUDE,LONGITUDE,XCOORD,YCOORD,WARD,EVENTID,MAR_ADDRESS,MAR_SCORE,MAJORINJURIES_BICYCLIST,MINORINJURIES_BICYCLIST,UNKNOWNINJURIES_BICYCLIST,FATAL_BICYCLIST,MAJORINJURIES_DRIVER,MINORINJURIES_DRIVER,UNKNOWNINJURIES_DRIVER,FATAL_DRIVER,MAJORINJURIES_PEDESTRIAN,MINORINJURIES_PEDESTRIAN,UNKNOWNINJURIES_PEDESTRIAN,FATAL_PEDESTRIAN,TOTAL_VEHICLES,TOTAL_BICYCLES,TOTAL_PEDESTRIANS,PEDESTRIANSIMPAIRED,BICYCLISTSIMPAIRED,DRIVERSIMPAIRED,TOTAL_TAXIS,TOTAL_GOVERNMENT,SPEEDING_INVOLVED,NEARESTINTROUTEID,NEARESTINTSTREETNAME,OFFINTERSECTION,INTAPPROACHDIRECTION,LOCATIONERROR,LASTUPDATEDATE,MPDLATITUDE,MPDLONGITUDE,MPDGEOX,MPDGEOY,BLOCKKEY,SUBBLOCKKEY
0,-76.980766,38.889685,109482909,27191710,17010515,2017-01-19T18:56:20.000Z,13001702,13.411,0.000000,5431.0,2398.0,2017-01-19T05:00:00.000Z,NaN,58951,21 17TH ST SE,38.889678,-76.980710,401668.840,135759.618,Ward 6,{9AC03520-FDDA-4D43-848E-95EAF2FC8E74},1635 EAST CAPITOL STREET SE,200.0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,12001702,17TH ST NE,14.334858,South,NaN,NaN,38.889678,-76.980710,NaN,NaN,NaN,NaN
1,-77.029993,38.952914,109482910,27688005,18085167,2018-05-27T05:47:21.000Z,11042902,25.472,19.612804,6921.0,9510.0,2018-05-26T04:00:00.000Z,NaN,254976,5200 13TH STREET NW,38.952907,-77.029990,397400.439,142778.768,Ward 4,{5B7C4496-EAFE-4681-BFE4-602357ADBA39},5200 13TH STREET NW,200.0,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,11001302,13TH ST NW,2.464102,West,NaN,2018-05-30T14:33:06.000Z,38.952885,-77.029683,397400.55,142798.38,NaN,NaN
2,-76.989326,38.912538,109482911,27412026,17147314,2017-08-26T04:18:45.000Z,47061782,6.668,1.181367,-9.0,28725.0,2017-08-25T04:00:00.000Z,NaN,285877,956 MOUNT OLIVET ROAD NE,38.912530,-76.989323,400925.959,138296.279,Ward 5,{AA7672E8-9AD6-4DF5-99A2-485C44F778AC},1125 OKIE STREET NE,200.0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,12062462,MOUNT OLIVET RD NE,81.113797,East,NaN,NaN,38.912540,-76.989318,400869.06,138300.03,NaN,NaN
3,-77.009782,38.825201,109482912,27191714,17010532,2017-01-19T19:01:35.000Z,14036982,263.347,0.000000,2270.0,1052.0,2017-01-19T05:00:00.000Z,NaN,82880,77 FORRESTER STREET SW,38.825274,-77.009778,399150.748,128601.264,Ward 8,{A4BB396B-2DAA-4132-AE90-3DBB53E718F4},95 FORRESTER STREET SW,200.0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,14039322,GALVESTON PL SW,172.288758,West,NaN,NaN,38.825274,-77.009778,399157.70,128623.88,NaN,NaN
4,-76.958041,38.920396,109482913,27191715,17010514,2017-01-19T19:01:37.000Z,12081442,372.161,0.000000,9851.0,36763.0,2017-01-19T05:00:00.000Z,NaN,304441,2200 SOUTH DAKOTA AVENUE NE,38.920464,-76.957954,403638.852,139169.372,Ward 5,{EF8064CF-B419-468F-B0D2-D5F777142E35},3320 THEODORE R HAGANS DRIVE NE,200.0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,12003332,33RD PL NE,56.061139,Northwest,NaN,NaN,38.920464,-76.957954,403599.77,139127.65,NaN,NaN


In [52]:
df_crashes.rename(columns=lambda x:x.lower(), inplace=True)

In [53]:
df_crashes.columns

Index(['x', 'y', 'objectid', 'crimeid', 'ccn', 'reportdate', 'routeid',
       'measure', 'offset', 'streetsegid', 'roadwaysegid', 'fromdate',
       'todate', 'marid', 'address', 'latitude', 'longitude', 'xcoord',
       'ycoord', 'ward', 'eventid', 'mar_address', 'mar_score',
       'majorinjuries_bicyclist', 'minorinjuries_bicyclist',
       'unknowninjuries_bicyclist', 'fatal_bicyclist', 'majorinjuries_driver',
       'minorinjuries_driver', 'unknowninjuries_driver', 'fatal_driver',
       'majorinjuries_pedestrian', 'minorinjuries_pedestrian',
       'unknowninjuries_pedestrian', 'fatal_pedestrian', 'total_vehicles',
       'total_bicycles', 'total_pedestrians', 'pedestriansimpaired',
       'bicyclistsimpaired', 'driversimpaired', 'total_taxis',
       'total_government', 'speeding_involved', 'nearestintrouteid',
       'nearestintstreetname', 'offintersection', 'intapproachdirection',
       'locationerror', 'lastupdatedate', 'mpdlatitude', 'mpdlongitude',
       'mpdgeox', 'mpd

In [54]:
df_crashes[df_crashes['fatal_bicyclist'] > 0]

,x,y,objectid,crimeid,ccn,reportdate,routeid,measure,offset,streetsegid,roadwaysegid,fromdate,todate,marid,address,latitude,longitude,xcoord,ycoord,ward,eventid,mar_address,mar_score,majorinjuries_bicyclist,minorinjuries_bicyclist,unknowninjuries_bicyclist,fatal_bicyclist,majorinjuries_driver,minorinjuries_driver,unknowninjuries_driver,fatal_driver,majorinjuries_pedestrian,minorinjuries_pedestrian,unknowninjuries_pedestrian,fatal_pedestrian,total_vehicles,total_bicycles,total_pedestrians,pedestriansimpaired,bicyclistsimpaired,driversimpaired,total_taxis,total_government,speeding_involved,nearestintrouteid,nearestintstreetname,offintersection,intapproachdirection,locationerror,lastupdatedate,mpdlatitude,mpdlongitude,mpdgeox,mpdgeoy,blockkey,subblockkey
22343,-77.042718,38.909397,109504252,27805791,18159474,2018-09-21T20:47:46.000Z,11031202,92.004,0.004569,8594.0,14269.0,2018-09-21T04:00:00.000Z,NaN,241264,DUPONT CIRCLE NW & MASSACHUSETTS AVENUE NW,38.909390,-77.042716,396295.124,137948.498,Ward 2,{20EA71FF-48A4-414F-918A-CA12E73DAFC6},1369 CONNECTICUT AVENUE NW,200.0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,11059602,MASSACHUSETTS AVE NW,1.379358,NaN,NaN,NaN,38.909894,-77.044153,396295.12,137948.50,NaN,NaN
24819,-77.028140,38.892121,109507728,27809518,18161337,2018-09-25T11:57:03.000Z,11001202,183.961,2.318473,12430.0,12718.0,2018-09-24T04:00:00.000Z,NaN,317203,1201 CONSTITUTION AVENUE NW,38.892114,-77.028137,397558.994,136030.223,Ward 2,{EE1C4682-8DEA-4AF0-8F7A-A38972B00121},210 12TH STREET NW,200.0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,11001242,12TH EXPY NW,3.924697,West,NaN,NaN,38.892114,-77.028164,NaN,NaN,NaN,NaN
42531,-76.993579,38.839128,109524440,26755738,15180926,2015-11-13T02:43:05.000Z,13092882,510.540,0.000000,3389.0,3957.0,2015-11-12T05:00:00.000Z,NaN,31978,3503 WHEELER RD SE,38.839118,-76.993686,400557.668,130147.169,Ward 8,{3D89A7FB-B209-4B38-AAAE-33A671116D0C},3701 WHEELER ROAD SE,200.0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,13061292,MISSISSIPPI AVE SE,36.407368,North,NaN,NaN,38.839118,-76.993686,400534.69,130206.38,NaN,NaN
63185,-77.025920,38.932026,109545094,24399981,13004994,2013-01-12T05:00:00.000Z,47032042,34.138,0.000000,-9.0,25669.0,2013-01-12T05:00:00.000Z,NaN,233033,765 MORTON ST NW,38.932019,-77.025918,397752.822,140459.953,Ward 1,{FD77F5F5-649C-4AA8-84A9-410839FD4977},765 MORTON STREET NW,100.0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,11062402,MORTON ST NW,20.487304,North,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71830,-77.027269,38.917008,109552739,24602397,13065228,2013-05-17T05:00:00.000Z,11087232,1190.244,0.000000,10458.0,6224.0,2013-05-16T04:00:00.000Z,NaN,305382,1100 U ST NW,38.916844,-77.027265,397635.383,138792.838,Ward 1,{8C70A403-8C7F-47D7-8CA1-B780C63DF599},1100 U STREET NW,200.0,0,0,0,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,11001102,11TH ST NW,26.251791,West,NaN,NaN,38.916844,-77.027265,NaN,NaN,NaN,NaN
90426,-77.022947,38.914097,109573335,25135028,14143805,2014-09-20T05:00:00.000Z,11000802,1177.442,0.000000,9716.0,14542.0,2014-09-20T04:00:00.000Z,NaN,906686,8TH STREET AND S STREET NW.,38.914089,-77.022945,398010.090,138469.530,Ward 1,{C655C431-9AD6-473C-BAB0-A4C75F00F910},S STREET NW AND 8TH STREET NW,87.5,0,0,0,1,0,0,0,0,0,0,0,0,4,1,0,0,0,0,0,0,0,11077672,S ST NW,0.038099,North,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138469,-76.990064,38.902924,109621378,28015914,19066417,2019-04-19T23:52:25.000Z,12061622,22.329,14.410425,9054.0,746.0,2019-04-19T04:00:00.000Z,NaN,72282,1200 FLORIDA AVENUE NE,38.902917,-76.990062,400862.013,137229.122,Ward 5,{0A8A6E3A-1CC4-4F00-B6C9-8EB4BE505A43},1200 FLORIDA AVENUE NE,200.0,0,0,0,1,1,0,0,0,0,1,0,0,2,1,1,0,0,0,0,0,1,12001202,12TH ST NE,24.966315,North,NaN,NaN,38.902785,-76.990465,400874.69,137222.27,NaN,NaN
174119,-77.001307,38.900212,109656028,27717508,18102675,2018-06-25T10:51:19.000Z,12042442,63.631,0.009074,3943.0,37261.0,2018-06-23T04:00:00.000Z,NaN,39304,H STREET NE,38.900205,-77.001305,399886.830,136928.019,Ward 6,{17E197E1-DBA0-4FB6-9A9D-25E2EDD5DF36},316 H STREE

In [55]:
df_crashes = df_crashes.loc[:, ['objectid', 'latitude', 'longitude', 'fatal_bicyclist','majorinjuries_bicyclist', 'minorinjuries_bicyclist',
                               'bicyclistsimpaired','reportdate', 'streetsegid', 'roadwaysegid','address',
                                 'ward', 'nearestintstreetname']]

In [56]:
df_crashes.head()

,objectid,latitude,longitude,fatal_bicyclist,majorinjuries_bicyclist,minorinjuries_bicyclist,bicyclistsimpaired,reportdate,streetsegid,roadwaysegid,address,ward,nearestintstreetname
0,109482909,38.889678,-76.980710,0,0,0,0,2017-01-19T18:56:20.000Z,5431.0,2398.0,21 17TH ST SE,Ward 6,17TH ST NE
1,109482910,38.952907,-77.029990,0,0,0,0,2018-05-27T05:47:21.000Z,6921.0,9510.0,5200 13TH STREET NW,Ward 4,13TH ST NW
2,109482911,38.912530,-76.989323,0,0,0,0,2017-08-26T04:18:45.000Z,-9.0,28725.0,956 MOUNT OLIVET ROAD NE,Ward 5,MOUNT OLIVET RD NE
3,109482912,38.825274,-77.009778,0,0,0,0,2017-01-19T19:01:35.000Z,2270.0,1052.0,77 FORRESTER STREET SW,Ward 8,GALVESTON PL SW
4,109482913,38.920464,-76.957954,0,0,0,0,2017-01-19T19:01:37.000Z,9851.0,36763.0,2200 SOUTH DAKOTA AVENUE NE,Ward 5,33RD PL NE


In [57]:
df_bike_crashes = df_crashes[(df_crashes['majorinjuries_bicyclist'] > 0) | (df_crashes['minorinjuries_bicyclist'] > 0) |
                              (df_crashes['bicyclistsimpaired'] > 0 ) | (df_crashes['fatal_bicyclist'] > 0) ]

In [58]:
df_bike_crashes.head()

,objectid,latitude,longitude,fatal_bicyclist,majorinjuries_bicyclist,minorinjuries_bicyclist,bicyclistsimpaired,reportdate,streetsegid,roadwaysegid,address,ward,nearestintstreetname
74,109482983,38.927501,-77.037145,0,0,1,0,2017-12-03T11:08:07.000Z,-9.0,28500.0,1600 HOBART STREET NW,Ward 1,MOUNT PLEASANT ST NW
87,109482996,38.911446,-77.048754,0,0,1,0,2017-12-03T16:09:56.000Z,8675.0,9223.0,EB MASS AVE NW & 22ND ST NW\r\nW,Ward 2,FLORIDA AVE NW
131,109483040,38.924756,-77.032242,0,0,1,0,2017-12-04T00:08:27.000Z,4017.0,13402.0,14TH STREET NW & FAIRMONT STREET NW,Ward 1,FAIRMONT ST NW
138,109483047,38.824860,-76.999634,0,0,1,0,2015-08-30T22:17:31.000Z,6057.0,13941.0,4275 4TH STREET SE,Ward 8,GALVESTON PL SE
205,109483114,38.929009,-77.037461,0,0,1,0,2018-08-11T18:10:20.000Z,2997.0,9215.0,3102 MOUNT PLEASANT STREET NW,Ward 1,IRVING ST NW


In [59]:
len(df_bike_crashes[df_bike_crashes['bicyclistsimpaired'] > 0 ])

21

In [60]:
# df_weather.head()

In [61]:
len(df_crashes)

217602

In [62]:
len(df_vision)

5566

## Looking at Bike-Specific Data (Crashes & Vision Zero)

In [63]:
len(df_bike_vision)

3420

In [64]:
len(df_bike_crashes)

2313

In [65]:
df_bike_crashes.columns

Index(['objectid', 'latitude', 'longitude', 'fatal_bicyclist',
       'majorinjuries_bicyclist', 'minorinjuries_bicyclist',
       'bicyclistsimpaired', 'reportdate', 'streetsegid', 'roadwaysegid',
       'address', 'ward', 'nearestintstreetname'],
      dtype='object')

In [66]:
df_bike_vision.columns

Index(['objectid', 'latitude', 'longitude', 'requesttype', 'requestdate',
       'streetsegid', 'comments', 'usertype'],
      dtype='object')

In [67]:
###If we would decide to round latitude and longitude for some reason, this code does that.
###########################################################################################

# round(df_vision['latitude'][1:15], 4)

# This rounds so that it is Degrees DD, Minutes MM, and Seconds SS

# df_vision['latitude'] = round(df_vision['latitude'],4)
# df_vision['longitude'] = round(df_vision['longitude'],4)

# df_crashes['latitude'] = round(df_crashes['latitude'],4)
# df_crashes['longitude'] = round(df_crashes['longitude'],4)

In [68]:
df_bike_crashes['streetsegid'].isna().value_counts(normalize=True)

False    0.890186
True     0.109814
Name: streetsegid, dtype: float64

In [69]:
len(df_bike_crashes[df_bike_crashes['streetsegid'] == -9]) / 2302

0.10903562119895743

In [70]:
df_bike_vision['streetsegid'].isna().value_counts()

False    3400
True       20
Name: streetsegid, dtype: int64

In [71]:
df_bike_crashes['streetsegid'].value_counts()

-9.0        251
 763.0       12
 10588.0     10
 1292.0       8
 0.0          8
 3684.0       6
 9543.0       6
 11313.0      6
 1562.0       6
 9137.0       5
 9179.0       5
 8575.0       5
 11932.0      5
 5270.0       5
 6079.0       5
 1100.0       5
 5274.0       5
 5540.0       5
 10495.0      5
 8191.0       4
 74.0         4
 5292.0       4
 12509.0      4
 7639.0       4
 4818.0       4
 12987.0      4
 7159.0       4
 13189.0      4
 6054.0       4
 985.0        4
 7453.0       4
 13135.0      4
 14208.0      4
 6255.0       4
 7676.0       4
 1841.0       4
 1103.0       4
 12913.0      4
 9054.0       3
 3004.0       3
 4079.0       3
 10290.0      3
 8634.0       3
 10134.0      3
 7452.0       3
 4389.0       3
 11676.0      3
 9819.0       3
 5121.0       3
 5137.0       3
 9726.0       3
 1681.0       3
 5811.0       3
 1333.0       3
 7725.0       3
 7491.0       3
 7336.0       3
 13216.0      3
 11347.0      3
 8124.0       3
 6775.0       3
 12067.0      3
 3140.0 

In [72]:
df_bike_crashes[df_bike_crashes['streetsegid'] == -9]

,objectid,latitude,longitude,fatal_bicyclist,majorinjuries_bicyclist,minorinjuries_bicyclist,bicyclistsimpaired,reportdate,streetsegid,roadwaysegid,address,ward,nearestintstreetname
74,109482983,38.927501,-77.037145,0,0,1,0,2017-12-03T11:08:07.000Z,-9.0,28500.0,1600 HOBART STREET NW,Ward 1,MOUNT PLEASANT ST NW
239,109483148,38.941229,-77.021211,0,0,1,0,2018-08-21T04:04:08.000Z,-9.0,24144.0,4106 NEW HAMPSHIRE AVENUE NW,Ward 4,7TH ST NW
555,109483464,38.920461,-77.021317,0,0,1,0,2018-08-20T15:05:57.000Z,-9.0,34773.0,2301 GEORGIA AVENUE NW,Ward 1,BRYANT ST NW
1261,109482170,38.927519,-77.004490,0,0,1,0,2019-02-08T02:17:54.000Z,-9.0,33567.0,125 MICHIGAN AVENUE NE,Ward 5,MICHIGAN AVE NE
1499,109482408,38.899063,-77.028228,0,0,1,0,2018-04-04T00:31:21.000Z,-9.0,27413.0,700 12TH STREET NW,Ward 2,12TH ST NW
4426,109487335,38.937379,-77.033233,0,0,1,0,2015-12-14T13:49:17.000Z,-9.0,28667.0,3708 14TH STREET NW,Ward 4,SPRING RD NW
4456,109487365,38.877369,-77.014366,0,0,1,0,2017-05-19T21:49:43.000Z,-9.0,33506.0,210 L STREET SW,Ward 6,L ST SW
4618,109487527,38.904774,-77.060461,0,0,1,0,2017-04-02T01:55:51.000Z,-9.0,29798.0,3040 M STREET NW,Ward 2,THOMAS JEFFERSON ST NW
5404,109486313,38.923243,-77.041006,0,0,1,0,2017-12-04T19:38:51.000Z,-9.0,28382.0,2503 CHAMPLAIN STREET NW,Ward 1,EUCLID ST NW
5561,109486470,38.905352,-76.999739,0,0,1,0,2018-08-24T17:34:38.000Z,-9.0,24650.0,1166 5TH STREET NE,Ward 6,5TH ST NE


## Writing cleaned data sets before beginning Merges

In [73]:
!pwd

/c/Users/602770/Downloads/wids/Raw Data


In [74]:
df_potholes.to_csv('../Processed Data/Potholes_Aug_to_Oct_19.csv', index=False)

In [75]:
#Safety Data Sets
df_bike_crashes.to_csv('../Processed Data/Bike_Crashes.csv', index=False)
df_bike_vision.to_csv('../Processed Data/Bike_Vision.csv', index=False)
df_potholes.to_csv('../Processed Data/Potholes_Aug_to_Oct_19.csv', index=False)
df_lights.to_csv('../Processed Data/Lights.csv', index=False)

#Bike Lane & Street Segment Data Sets
df_bikelane.to_csv('../Processed Data/Bike_Lane.csv', index=False)
df_ward.to_csv('../Processed Data/Ward.csv', index=False)

#Leaving in all columns, as it might be useful in connecting with OSRM coordinate data)
df_segments.to_csv('../Processed Data/Street_Segments.csv', index=False)


In [76]:
df_segments.columns

Index(['STREETSEGID', 'FACILITYID', 'SOURCEID', 'STREETID', 'REGISTEREDNAME',
       'STREETTYPE', 'QUADRANT', 'DIRECTIONALITY', 'SEGMENTTYPE', 'FROMNODEID',
       'TONODEID', 'FROMADDRESSLEFTTHEO', 'TOADDRESSLEFTTHEO',
       'FROMADDRESSRIGHTTHEO', 'TOADDRESSRIGHTTHEO', 'BEGINMEASURE',
       'ENDMEASURE', 'UPDATETIMESTAMP', 'OBJECTID_1', 'OBJECTID', 'SHAPELEN'],
      dtype='object')

In [77]:
df_segments.head()

,STREETSEGID,FACILITYID,SOURCEID,STREETID,REGISTEREDNAME,STREETTYPE,QUADRANT,DIRECTIONALITY,SEGMENTTYPE,FROMNODEID,TONODEID,FROMADDRESSLEFTTHEO,TOADDRESSLEFTTHEO,FROMADDRESSRIGHTTHEO,TOADDRESSRIGHTTHEO,BEGINMEASURE,ENDMEASURE,UPDATETIMESTAMP,OBJECTID_1,OBJECTID,SHAPELEN
0,7581,SEGID-7581,10180320,100180,18TH,ST,NW,2,2,21821,21704,3100.0,3154.0,3101.0,3155.0,0,97.487740,2005-11-01T12:55:47.000Z,2546788,2684322,97.487740
1,5497,SEGID-5497,10180330,100180,18TH,ST,NW,2,2,21704,21810,3156.0,3198.0,3157.0,3199.0,0,97.562354,2005-11-01T12:55:47.000Z,2546789,2684323,97.562354
2,8038,SEGID-8038,10180340,100180,18TH,ST,NW,2,2,21810,18209,3200.0,3298.0,3201.0,3299.0,0,101.945998,2005-11-01T12:55:47.000Z,2546790,2684324,101.946006
3,588,SEGID-588,10180350,100180,18TH,ST,NW,2,2,18209,21785,3300.0,3334.0,3301.0,3335.0,0,101.815721,2005-11-01T12:55:47.000Z,2546791,2684325,101.815713
4,9760,SEGID-9760,10180360,100180,18TH,ST,NW,2,2,21785,15677,3336.0,3398.0,3337.0,3399.0,0,97.960598,2005-11-01T12:55:47.000Z,2546792,2684326,97.960628
